In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import re
from multiprocessing import Pool
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
documents = pd.read_csv('../DATA/DATA.csv')
documents

,INDEX,TEXT
0,A001-00,Auto 001/00\n\nDEBIDO PROCESO DE TUTELA-Notifi...
1,A001-01,Auto 001/01\n\nNULIDAD SENTENCIA DE CONSTITUCI...
2,A001-02,Auto 001/02\n \nLEGITIMACION POR PASIVA EN TUT...
3,A001-03,Auto 001/03\n\nREFERENDO-Etapas de desarrollo ...
4,A001-04,Auto 001/04\n\nCONFLICTO DE COMPETENCIA EN TUT...
...,...,...
35732,T-999-08,Sentencia T-999/08\n\nDERECHO A LA SALUD COMO ...
35733,T-999-10,Sentencia T-999/10\n\nMUJER EMBARAZADA-Deber d...
35734,T-999-12,Sentencia T-999/12\n\n\nACCION DE TUTELA DE PE...
35735,T-999-99,Sentencia T-999/99\n\nDERECHO A LA SEGURIDAD S...


In [3]:
def preprocessor_sentences(text):
    """
    preprocesador de texto con separación de frases por línea.
    """
    text = text.replace("\n", " ")
    text = text.replace("\xa0", " ")
    text = text.replace("\x93", " ")
    text = text.replace("\x94", " ")
    text = text.replace("\r", " ")
    text = text.replace("\t", " ")
    return text

documents["TEXT_PLAIN"] = [preprocessor_sentences(str(x)) for x in tqdm(documents.TEXT)]

100%|██████████| 35737/35737 [00:02<00:00, 14254.92it/s]


In [4]:
documents.TEXT_PLAIN = [x.lower() for x in documents.TEXT_PLAIN]

In [5]:
documents

,INDEX,TEXT,TEXT_PLAIN
0,A001-00,Auto 001/00\n\nDEBIDO PROCESO DE TUTELA-Notifi...,auto 001/00 debido proceso de tutela-notifica...
1,A001-01,Auto 001/01\n\nNULIDAD SENTENCIA DE CONSTITUCI...,auto 001/01 nulidad sentencia de constitucion...
2,A001-02,Auto 001/02\n \nLEGITIMACION POR PASIVA EN TUT...,auto 001/02 legitimacion por pasiva en tutel...
3,A001-03,Auto 001/03\n\nREFERENDO-Etapas de desarrollo ...,auto 001/03 referendo-etapas de desarrollo de...
4,A001-04,Auto 001/04\n\nCONFLICTO DE COMPETENCIA EN TUT...,auto 001/04 conflicto de competencia en tutel...
...,...,...,...
35732,T-999-08,Sentencia T-999/08\n\nDERECHO A LA SALUD COMO ...,sentencia t-999/08 derecho a la salud como ga...
35733,T-999-10,Sentencia T-999/10\n\nMUJER EMBARAZADA-Deber d...,sentencia t-999/10 mujer embarazada-deber del...
35734,T-999-12,Sentencia T-999/12\n\n\nACCION DE TUTELA DE PE...,sentencia t-999/12 accion de tutela de perso...
35735,T-999-99,Sentencia T-999/99\n\nDERECHO A LA SEGURIDAD S...,sentencia t-999/99 derecho a la seguridad soc...


Magistrado Ponente: x
Magistrada ponente: x

Magistrada sustanciador: x
Magistrada sustanciadora: x

Solicitante: x
Demandante: x

Peticionario:

Procedencia: 

Actora:

# Magistrados ponentes:

In [6]:
#Magistrado Ponente:
search_word = 'Magistrado Ponente:'
search_word_2 = 'Magistrada ponente:'

magistrados_ponentes = []

count = 0
for document in tqdm(documents.TEXT_PLAIN):
    if search_word.lower() in document:
        #print(document.find(search_word.lower()))
        try:
            found = re.search(search_word.lower() + '(.+?)  ', document).group(1).replace("dr.", "").lstrip()
            
            if len(found) < 5:
                try:
                    found = re.search(search_word.lower() + '(.+?)bogo', document).group(1).replace("dr.", "").lstrip()
                except AttributeError:
                    found = '' 
            
        except AttributeError:
            found = '' 
    
    elif search_word_2.lower() in document:
        #print(document.find(search_word.lower()))
        try:
            found = re.search(search_word_2.lower() + '(.+?)  ', document).group(1).replace("dr.", "").lstrip()
            
            if len(found) < 5:
                try:
                    found = re.search(search_word_2.lower() + '(.+?)bogo', document).group(1).replace("dr.", "").lstrip()
                except AttributeError:
                    found = '' 
            
        except AttributeError:
            found = '' 
    
    else:
        found = ''
       
    if len(found)<40 and len(found)>5:
        #print(found)
        #print(len(found))
        #print('*'*100)
        magistrados_ponentes.append(found)
        count += 1
    else: 
        magistrados_ponentes.append('')

100%|██████████| 35737/35737 [00:01<00:00, 26829.59it/s]


In [7]:
count/len(documents)

0.8604247698463777

In [8]:
len(magistrados_ponentes)

35737

In [108]:
magistrados_ponentes[0:100]

['',
 '',
 'manuel josé cepeda espinosa',
 '',
 'marco gerardo monroy cabra',
 'marco gerardo monroy cabra',
 'alfredo beltrán sierra.',
 'manuel josé cepeda espinosa',
 'marco gerardo monroy cabra',
 'nilson pinilla pinilla',
 '',
 'mauricio gonzález cuervo.',
 'juan carlos henao perez',
 'luis ernesto vargas silva.   ',
 'jorge ignacio pretelt chaljub',
 'alberto rojas ríos',
 '',
 '',
 'diana fajardo rivera',
 'alberto rojas ríos',
 'paola andrea meneses mosquera',
 '',
 '',
 'hernando herrera vergara',
 '',
 'carlos gaviria díaz.',
 'antonio barrera carbonell',
 'antonio barrera carbonell',
 'alfredo beltrán sierra',
 'dra. clara inés vargas hernández',
 'cristina pardo schlesinger',
 '',
 '',
 '',
 'carlos gaviria díaz',
 '',
 'marco gerardo monroy cabra',
 '',
 'jaime córdoba triviño',
 'jaime córdoba triviño',
 'alfredo beltrán sierra',
 'manuel josé cepeda espinosa',
 'manuel josé cepeda espinosa',
 'mauricio gonzalez cuervo.',
 'mauricio gonzález cuervo',
 '',
 'mauricio gonzá

# Magistrados sustanciadores

In [9]:
#Magistrado sutanciador:
search_word = 'Magistrado sustanciador:'
search_word_2 = 'Magistrada sustanciadora:'

magistrados_sustanciadores = []

count = 0
for document in tqdm(documents.TEXT_PLAIN):
    if search_word.lower() in document:
        #print(document.find(search_word.lower()))
        try:
            found = re.search(search_word.lower() + '(.+?)  ', document).group(1).replace("dr.", "").lstrip()
            
            if len(found) < 5:
                try:
                    found = re.search(search_word.lower() + '(.+?)bogo', document).group(1).replace("dr.", "").lstrip()
                except AttributeError:
                    found = '' 
            
        except AttributeError:
            found = '' 
    
    elif search_word_2.lower() in document:
        #print(document.find(search_word.lower()))
        try:
            found = re.search(search_word_2.lower() + '(.+?)  ', document).group(1).replace("dr.", "").lstrip()
            
            if len(found) < 5:
                try:
                    found = re.search(search_word_2.lower() + '(.+?)bogo', document).group(1).replace("dr.", "").lstrip()
                except AttributeError:
                    found = '' 
            
        except AttributeError:
            found = '' 
    
    else:
        found = ''
       
    if len(found)<40 and len(found)>5:
        #print(found)
        #print(len(found))
        #print('*'*100)
        magistrados_sustanciadores.append(found)
        count += 1
    else: 
        magistrados_sustanciadores.append('')

100%|██████████| 35737/35737 [00:02<00:00, 14003.37it/s]


In [10]:
count/len(documents)

0.0845622184290791

In [11]:
len(magistrados_sustanciadores)

35737

In [107]:
magistrados_sustanciadores[0:100]

['josé gregorio hernández galindo',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'jorge iván palacio palacio',
 '',
 '',
 '',
 '',
 '',
 'jose gregorio hernandez galindo',
 '',
 'antonio barrera carbonell',
 '',
 '',
 '',
 '',
 '',
 '',
 'jorge arango mejia',
 '',
 'jorge arango mejia',
 '',
 'carlos gaviria díaz.',
 '',
 'alfredo beltrán sierra',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'gloria stella ortiz delgado',
 '',
 '',
 '',
 '',
 '',
 'doctor jose gregorio hernandez galindo',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'fabio moron diaz',
 '',
 '',
 '',
 '',
 'alvaro tafur galvis',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'jorge ignacio pretelt chaljub',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'hernando herrera vergara    santafe de ',
 '',
 '',
 '',
 '',
 'vladimiro naranjo mesa',
 '',
 '',
 'vladimiro naranjo mesa.',
 '',
 '',
 'alfredo beltrán sierra',
 '']

# solicitantes y/o demandantes

In [15]:
search_word = 'Solicitante:'
search_word_2 = 'Demandante:'


solicitante_demandante = []
count = 0


for document in tqdm(documents.TEXT_PLAIN):
    if search_word.lower() in document:
        #print(document.find(search_word.lower()))
        try:
            found = re.search(search_word.lower() + '(.+?)  ', document).group(1).lstrip()
            
        except AttributeError:
            found = '' 

    elif search_word_2.lower() in document:
        #print(document.find(search_word.lower()))
        try:
            found = re.search(search_word_2.lower() + '(.+?)  ', document).group(1).lstrip()
            
        except AttributeError:
            found = '' 
    
    else:
        found = ''
       
    if len(found)<100 and len(found)>5:
        #print(found)
        #print(len(found))
        #print('*'*100)
        solicitante_demandante.append(found)
        count += 1
    else: 
        solicitante_demandante.append('')





100%|██████████| 35737/35737 [00:03<00:00, 10458.61it/s]


In [16]:
count

2949

In [17]:
len(solicitante_demandante)

35737

In [106]:
solicitante_demandante[0:100]

['',
 '',
 '',
 '',
 '',
 '',
 '',
 'ligia amparo torres acevedo',
 '',
 '',
 '',
 'nikitus trading ltd.',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'andrés sierra amazo',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'mauricio plazas vega',
 '',
 'julián cifuentes bolívar',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'eduardo afanador pico.',
 'juan manuel de castells y tejón',
 '']

# Extraccion de la fecha

In [78]:
import datetime

In [97]:
search_word = 'bogotá d.c.,'
search_word_3 = 'bogotá d. c.,'
search_word_2 = 'Bogotá, D. C.,'
search_word_4 = 'Bogotá, D.C.,'
search_word_5 = 'Bogotá,D.C.,'
search_word_6 = 'Bogotá, D. C. ,'
search_word_7 = 'Bogotá, D.C. ,'

meses = ['enero', 'febrero', 'marzo', 'abril', 'mayo', 'junio', 'julio', 'agosto', 'septiembre', 'octubre', 'noviembre', 'diciembre']

count = 0
dates = []

def date_finder_1(search_word, document):
    
    location = document.find(search_word.lower())
    date_raw = document[location: location+ 150]
    numbers = re.findall(r'\d+',date_raw)

    try:
        if numbers[1] == '1' and len(numbers[2])==3:
            numbers[1] = numbers[1] + numbers[2]
    except:
        pass
    try:
        if numbers[1] == '2' and len(numbers[2])==3:
            numbers[1] = numbers[1] + numbers[2]
    except:
        pass

    if len(numbers)>1:
        
        if 0<int(numbers[0])<32 and 1990<int(numbers[1])<2030:

            mes = ''

            for m in meses:
                if m in date_raw:
                    mes = m
                    break
            dia = numbers[0]
            año = numbers[1]

            if mes != '':
                
                try:
                    
                    document_date = datetime.datetime(int(año), meses.index(mes)+1, int(dia))
                    
                except Exception as e:
                    
                    print(document[0:20])
                    print(document[ document.find(search_word.lower()): document.find(search_word.lower())+100])
                    print('*'*10)
                    print(año)
                    print(mes)
                    print(dia)
                    print(e)
                    print('*'*100)
                    document_date = None
                    
                    
                return document_date

    
    

for document in tqdm(documents.TEXT_PLAIN):
    
    document = re.sub("\s\s+" , " ", document)
    
    if search_word.lower() in document:
        
        """
        
        location = document.find(search_word.lower())
        date_raw = document[location: location+ 150]
        numbers = re.findall(r'\d+',date_raw)
        
        try:
            if numbers[1] == '1' and len(numbers[2])==3:
                numbers[1] = numbers[1] + numbers[2]
        except:
            pass
        try:
            if numbers[1] == '2' and len(numbers[2])==3:
                numbers[1] = numbers[1] + numbers[2]
        except:
            pass
        
        if len(numbers)>1:
            if 0<int(numbers[0])<32 and 1990<int(numbers[1])<2030:
                
                mes = ''
                
                for m in meses:
                    if m in date_raw:
                        mes = m
                        break
                dia = numbers[0]
                año = numbers[1]
                
                if mes != '':
            
                    document_date = datetime.datetime(int(año), meses.index(mes)+1, int(dia))
                    dates.append(document_date)
                    
                    print(date_raw)
                    print(document_date)
                    print('*'*100)

                    count += 1
                    
        """

        
        document_date = date_finder_1(search_word,document)

        
        if document_date != None:
            dates.append(document_date)
            count +=1
            
        else:
            dates.append(document_date)
        
        
    elif search_word_2.lower() in document:
        #print(document.find(search_word.lower()))
        
        document_date = date_finder_1(search_word_2,document)

        if document_date != None:
            dates.append(document_date)
            count +=1
        else:
            dates.append(document_date)
        
   
  
    elif search_word_3.lower() in document:
        #print(document.find(search_word.lower()))
        
        document_date = date_finder_1(search_word_3,document)

        if document_date != None:
            dates.append(document_date)
            count +=1
        else:
            dates.append(document_date)
                    
    elif search_word_4.lower() in document:
        #print(document.find(search_word.lower()))
       
        document_date = date_finder_1(search_word_4,document)

        if document_date != None:
            dates.append(document_date)
            count +=1
        else:
            dates.append(document_date)
        
    elif search_word_5.lower() in document:
        #print(document.find(search_word.lower()))
       
        document_date = date_finder_1(search_word_5,document)

        if document_date != None:
            dates.append(document_date)
            count +=1
        else:
            dates.append(document_date)
        
        
    elif search_word_6.lower() in document:
        #print(document.find(search_word.lower()))
        
        document_date = date_finder_1(search_word_6,document)

        if document_date != None:
            dates.append(document_date)
            count +=1
        else:
            dates.append(document_date)
        
        
        
    elif search_word_7.lower() in document:
        #print(document.find(search_word.lower()))
       
        document_date = date_finder_1(search_word_7,document)

        if document_date != None:
            dates.append(document_date)
            count +=1
        else:
            dates.append(document_date)
    
    else:
        
        document_date = None
        dates.append(document_date)
    


 28%|██▊       | 10062/35737 [00:08<01:03, 405.51it/s]

sentencia c-1033/03 
bogotá, d. c., a 30 de marzo de 2000 señor embajador: tengo el honor de referirme a su nota 83/200 d
**********
2000
febrero
30
day is out of range for month
****************************************************************************************************


100%|██████████| 35737/35737 [00:51<00:00, 693.70it/s]


In [103]:
count

30831

In [104]:
len(dates)

35737

In [105]:
dates[0:100]

[datetime.datetime(2000, 1, 12, 0, 0),
 None,
 datetime.datetime(2002, 1, 24, 0, 0),
 datetime.datetime(2003, 1, 20, 0, 0),
 datetime.datetime(2004, 1, 20, 0, 0),
 datetime.datetime(2005, 1, 20, 0, 0),
 datetime.datetime(2006, 1, 19, 0, 0),
 datetime.datetime(2007, 1, 19, 0, 0),
 None,
 datetime.datetime(2009, 1, 13, 0, 0),
 datetime.datetime(2010, 1, 14, 0, 0),
 None,
 datetime.datetime(2012, 1, 24, 0, 0),
 None,
 datetime.datetime(2015, 1, 21, 0, 0),
 datetime.datetime(2016, 1, 13, 0, 0),
 datetime.datetime(2017, 1, 13, 0, 0),
 datetime.datetime(2018, 1, 22, 0, 0),
 datetime.datetime(2019, 1, 21, 0, 0),
 datetime.datetime(2020, 1, 14, 0, 0),
 datetime.datetime(2021, 1, 13, 0, 0),
 None,
 None,
 None,
 None,
 datetime.datetime(1996, 2, 7, 0, 0),
 None,
 datetime.datetime(1998, 1, 16, 0, 0),
 datetime.datetime(1999, 1, 21, 0, 0),
 datetime.datetime(2004, 1, 20, 0, 0),
 datetime.datetime(2021, 1, 12, 0, 0),
 None,
 None,
 None,
 datetime.datetime(2000, 1, 19, 0, 0),
 datetime.datetime(2

In [113]:
nombres_y_fechas = pd.DataFrame()
nombres_y_fechas = documents[['INDEX']].copy()
nombres_y_fechas['FECHA'] = dates
nombres_y_fechas['MAGISTRADOS_PONENTES'] = magistrados_ponentes
nombres_y_fechas['MAGISTRADOS_SUSTANCIADORES'] = magistrados_sustanciadores
nombres_y_fechas['SOLICITANTES_DEMANDANTES'] = solicitante_demandante


In [123]:
nombres_y_fechas.iloc[0:30]

,INDEX,FECHA,MAGISTRADOS_PONENTES,MAGISTRADOS_SUSTANCIADORES,SOLICITANTES_DEMANDANTES
0,A001-00,2000-01-12,,josé gregorio hernández galindo,
1,A001-01,NaT,,,
2,A001-02,2002-01-24,manuel josé cepeda espinosa,,
3,A001-03,2003-01-20,,,
4,A001-04,2004-01-20,marco gerardo monroy cabra,,
5,A001-05,2005-01-20,marco gerardo monroy cabra,,
6,A001-06,2006-01-19,alfredo beltrán sierra.,,
7,A001-07,2007-01-19,manuel josé cepeda espinosa,,ligia amparo torres acevedo
8,A001-08,NaT,marco gerardo monroy cabra,,
9,A001-09,2009-01-13,nilson pinilla pinilla,,


In [124]:
file_name = 'nombres_y_fechas_V1_0.csv'
nombres_y_fechas.to_csv(file_name, encoding='utf-8', index= False)